In [1]:
import xmltodict
import json
from collections import defaultdict
import pandas as pd
import numpy as np
import folium

In [2]:
with open('Campground.XML', 'r') as f:
    campground = xmltodict.parse(f.read())
with open('CampingFeaturesOptions.XML', 'r') as f:
    camping_features_options = xmltodict.parse(f.read())
with open('CampingTrailAssoc.XML', 'r') as f:
    camping_trail_assoc = xmltodict.parse(f.read())
with open('County.XML', 'r') as f:
    county = xmltodict.parse(f.read())
with open('Features.XML', 'r') as f:
    features = xmltodict.parse(f.read())
with open('Trail.XML', 'r') as f:
    trail = xmltodict.parse(f.read())
with open('TrailCounties.XML', 'r') as f:
    trail_counties = xmltodict.parse(f.read())
with open('TrailFeaturesOptions.XML', 'r') as f:
    trail_features_options = xmltodict.parse(f.read())
with open('TrailSurface.XML', 'r') as f:
    trail_surface = xmltodict.parse(f.read())

In [3]:
campground_json = json.dumps(campground)
camping_features_options_json = json.dumps(camping_features_options)
camping_trail_assoc_json = json.dumps(camping_trail_assoc)
county_json = json.dumps(county)
features_json = json.dumps(features)
trail_json = json.dumps(trail)
trail_counties_json = json.dumps(trail_counties)
trail_features_options_json = json.dumps(trail_features_options)
trail_surface_json = json.dumps(trail_surface)

In [4]:
campground_d = json.loads(campground_json)
camping_features_options_d = json.loads(camping_features_options_json)
camping_trail_assoc_d = json.loads(camping_trail_assoc_json)
county_d = json.loads(county_json)
features_d = json.loads(features_json)
trail_d = json.loads(trail_json)
trail_counties_d = json.loads(trail_counties_json)
trail_features_options_d = json.loads(trail_features_options_json)
trail_surface_d = json.loads(trail_surface_json)

In [5]:
def clean_keys(data):
    """
    Removes the '@' character from the keys in a dictionary, recursively.

    Args:
    data: a dictionary or list to clean

    Returns:
    A dictionary or list with the '@' character removed from all keys.
    """

    if isinstance(data, list):
        return [clean_keys(item) for item in data]
    elif isinstance(data, dict):
        return {key.lstrip('@'): clean_keys(value) for key, value in data.items()}
    else:
        return data

In [6]:
campground_d = clean_keys(campground_d)
camping_features_options_d = clean_keys(camping_features_options_d)
camping_trail_assoc_d = clean_keys(camping_trail_assoc_d)
county_d = clean_keys(county_d)
features_d = clean_keys(features_d)
trail_d = clean_keys(trail_d)
trail_counties_d = clean_keys(trail_counties_d)
trail_features_options_d = clean_keys(trail_features_options_d)
trail_surface_d = clean_keys(trail_surface_d)

In [7]:
class TreeNode:
    """
    A class representing a node in a tree data structure.
    """

    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, child):
        """
        Adds a child node to the current node.

        Args:
        child: the child node to add
        """

        self.children.append(child)

    def display(self, level=0):
        """
        Prints the data of the current node and its children, indented according to their depth.

        Args:
        level: the depth of the current node (used for indentation)
        """

        print("  " * level + str(self.data))
        for child in self.children:
            child.display(level + 1)

In [8]:
def create_tree_structure(county_dict, trail_counties_dict, trail_dict, campground_dict, camping_trail_assoc_dict, camping_features_options_dict, features_dict):
    """
    Creates a tree structure representing Michigan's counties, trails, campgrounds, and features.

    Args:
    county_dict: a dictionary containing information about Michigan's counties
    trail_counties_dict: a dictionary containing information about the counties through which Michigan's trails pass
    trail_dict: a dictionary containing information about Michigan's trails
    campground_dict: a dictionary containing information about Michigan's campgrounds
    camping_trail_assoc_dict: a dictionary containing information about the association between Michigan's campgrounds and trails
    camping_features_options_dict: a dictionary containing information about the features available at Michigan's campgrounds
    features_dict: a dictionary containing information about the features available at Michigan's trails

    Returns:
    A TreeNode object representing the tree structure.
    """

    county_data = county_dict['ROOT']['dbo.County']
    trail_counties_data = trail_counties_dict['ROOT']['dbo.TrailCounties_Export_v']
    trail_data = trail_dict['ROOT']['dbo.Trail_Export_v']
    campground_data = campground_dict['ROOT']['dbo.Camping_Export_v']
    camping_trail_assoc_data = camping_trail_assoc_dict['ROOT']['dbo.CampingTrailAssoc_Export_v']
    camping_features_options_data = camping_features_options_dict['ROOT']['dbo.CampingFeaturesOptions_Export_v']
    features_data = features_dict['ROOT']['dbo.Features_Export_v']

    michigan_node = TreeNode("Michigan")
    county_nodes = {}
    trail_nodes = {}
    campground_nodes = {}
    feature_nodes = {}

    trail_id_to_trail = {trail['TrailId']: trail for trail in trail_data}
    camp_id_to_campground = {camp['CampID']: camp for camp in campground_data}
    feature_id_to_features = {feature['FeaturesID']: feature for feature in features_data}

    for county in county_data:
        county_node = TreeNode(county)
        county_nodes[county['Cntycode']] = county_node
        michigan_node.add_child(county_node)

    for trail_county in trail_counties_data:
        county_code = trail_county['CountyCode']
        trail_id = trail_county['TrailID']
        if county_code in county_nodes:
            county_node = county_nodes[county_code]
            if trail_id not in trail_nodes and trail_id in trail_id_to_trail:
                trail_node = TreeNode(trail_id_to_trail[trail_id])
                trail_nodes[trail_id] = trail_node
                county_node.add_child(trail_node)

    for assoc in camping_trail_assoc_data:
        trail_id = assoc['TrailID']
        camp_id = assoc['CampID']
        if trail_id in trail_nodes:
            trail_node = trail_nodes[trail_id]
            if camp_id in camp_id_to_campground:
                campground_node = TreeNode(camp_id_to_campground[camp_id])
                campground_nodes[camp_id] = campground_node
                trail_node.add_child(campground_node)

    for camp_feature in camping_features_options_data:
        camp_id = camp_feature['CampID']
        feature_id = camp_feature['FeatureID']
        if camp_id in campground_nodes:
            campground_node = campground_nodes[camp_id]
            if feature_id in feature_id_to_features:
                feature_node = TreeNode(feature_id_to_features[feature_id])
                feature_nodes[feature_id] = feature_node
                campground_node.add_child(feature_node)

    return michigan_node

In [9]:
michigan_tree = create_tree_structure(county_d, trail_counties_d, trail_d, campground_d, camping_trail_assoc_d, camping_features_options_d, features_d)
michigan_tree.display()

Michigan
  {'Cntycode': '01', 'Cntyabbr': 'ALCON', 'Cntyname': 'ALCONA', 'Cnamemixed': 'Alcona', 'Landarea_Acres': '433920', 'Landarea_Sqmi': '678', 'Region': '2', 'Pprism_Cntycode': '01', 'Fips_Cntycode': '26001', 'NCC_Cntycode': '001', 'Lic_Cntycode': '33'}
    {'TrailId': '120', 'Active': '1', 'TrailName': 'Harrisville-Cedar Run Nature Trail', 'TrailLength': '2.00', 'ORVVehicleType': '0', 'ShoreToShoreLength': '0', 'PredomTrailSurface': '1', 'PredomLength': '0', '_2ndTrailSurface': '0', 'UnitID': '0', 'Trailmap': 'harrisville_map.pdf', 'TrailPhoneNum': '(989) 724-5126', 'ADA': '0', 'LastUpdateTimestamp': '2019-03-20T14:37:47.263', 'DisplayHeaderGIF': 'Harrisvillecedrntr.gif', 'LengthyDescription': 'The Harrisville-Cedar Run Nature Trail is located in Harrisville State Park, one of the state?s oldest state parks. The park contains more than 100 acres of heavily forested land, as well as a mile of sandy beach along Lake Huron. The Cedar Run Nature Trail winds for two miles through the

In [10]:
def extract_coordinates_and_info(tree_node, coordinates, info):
    """
    Extracts the coordinates and information of nodes in a tree structure and stores them in lists.
    Args:
    tree_node: the root node of the tree structure to extract from
    coordinates: a list to store the coordinates of the nodes
    info: a list to store the information of the nodes
    """

    data = tree_node.data
    if "Lat" in data and "Long" in data:
        lat, lon = float(data["Lat"]), float(data["Long"])
        coordinates.append((lat, lon))
        info.append(data)
    for child in tree_node.children:
        extract_coordinates_and_info(child, coordinates, info)

coordinates = []
info = []
extract_coordinates_and_info(michigan_tree, coordinates, info)

In [11]:
def generate_map(coordinates, info):
    """
    Generates a folium map using the given coordinates and information.

    Args:
    coordinates: a list of coordinates to plot on the map
    info: a list of information about the coordinates

    Returns:
    A folium map object.
    """

    michigan_map = folium.Map(location=[43.621195, -84.682435], zoom_start=7)

    for coord, details in zip(coordinates, info):
        lat, lon = coord
        if "TrailName" in details:
            park_trail = int(details['ParkTrail'])
            forest_pathway = int(details['ForestPathway'])
            trail_type = "Park Trail" if park_trail else "Forest Pathway" if forest_pathway else "Unknown"
            icon_color = "green" if park_trail else "orange" if forest_pathway else "gray"
            popup_text = f"Trail: {details['TrailName']}<br>Type: {trail_type}<br>Length: {details.get('TrailLength', 'N/A')} miles"
            icon = folium.Icon(color=icon_color, icon="tree", prefix="fa")
        elif "Name" in details:
            campground_types = ['ADA', 'Rustic', 'Modern', 'SemiModern', 'Cabin', 'Equestrian', 'TeePee', 'Yurt', 'Lodge', 'MiniCabin']
            campground_type = [t for t in campground_types if details[t] == '1']
            campground_type_str = ', '.join(campground_type) if campground_type else "Unknown"
            icon_color = "blue"
            popup_text = f"Campground: {details['Name']}<br>Type: {campground_type_str}"
            icon = folium.Icon(color=icon_color, icon="campground", prefix="fa")
        else:
            continue

        marker = folium.Marker(location=[lat, lon], popup=popup_text, icon=icon)
        marker.add_to(michigan_map)

    return michigan_map

In [12]:
map = generate_map(coordinates, info)
map.save("michigan_map.html")

In [16]:
def filter_data_by_county(tree_node, county_name, filtered_coordinates, filtered_info):
    """
    Filters the data in a tree structure by county and stores the filtered coordinates and information in lists.

    Args:
    tree_node: the root node of the tree structure to filter
    county_name: the name of the county to filter by
    filtered_coordinates: a list to store the filtered coordinates
    filtered_info: a list to store the filtered information
    """

    if "Cntyname" in tree_node.data and tree_node.data["Cntyname"] == county_name.upper():
        extract_coordinates_and_info(tree_node, filtered_coordinates, filtered_info)

    for child in tree_node.children:
        filter_data_by_county(child, county_name, filtered_coordinates, filtered_info)

    return filtered_coordinates, filtered_info

county_name = "Washtenaw"
filtered_coordinates, filtered_info = [], []
filter_data_by_county(michigan_tree, county_name, filtered_coordinates, filtered_info)
filtered_map = generate_map(filtered_coordinates, filtered_info)
filtered_map.save(f"{county_name}_map.html")